# Aggression

In [1]:
! pip install transformers -qqq

In [3]:
import transformers
import numpy as np
import nltk
import pandas as pd

# nltk setup
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
def create_aug_pipeline(model_name : str):
  return transformers.pipeline("fill-mask", model_name)

aug_pipe = create_aug_pipeline("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
def stringCleanerMasker(ip_string, stop_words, num_mask = 1, mask_delim = '[MASK]'):

  ip_list = [tok for tok in ip_string.split() if tok not in stop_words]
  length = len(ip_list)

  if length > 512:
    return 'INVALID'

  mask_token = np.random.choice(ip_list, num_mask)[0]

  # print("\n MASK  : ", str(mask_token))

  finalString = ' '.join(ip_list)

  finalString = finalString.replace(str(mask_token), mask_delim, 1)

  # print("\n STRING_MODIFIED ", ip_string)

  return finalString

In [34]:
train = pd.read_csv('https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/trac2_eng_train.csv')
val = pd.read_csv('https://raw.githubusercontent.com/Dutta-SD/NLP/master/Aggression_Detection/trac2_eng_dev.csv')
train.head()
train.drop(['ID', 'Sub-task A'], axis=  1, inplace = True)
val.drop(['ID', 'Sub-task A'], axis = 1, inplace = True)
train.tail(10)

,Text,Sub-task B
4253,Ban suedo femists and liberals ...F..off,NGEN
4254,Open bob and vagane,GEN
4255,"With all due respect sir, I don't agree with y...",NGEN
4256,Sardaror sikhs we respect because of their pat...,NGEN
4257,Why so many youtube ads in your chanel ?,NGEN
4258,Abey loudey Arnab... Did u ever see the vedios...,GEN
4259,Arundati is very rich where she get money who ...,NGEN
4260,People may criticize Pratik Borade by saying t...,NGEN
4261,@Naaz Sk hello,NGEN
4262,"We want to read your book sir, please make it ...",NGEN


# Aggression Task (Task - 1) & 2


In [35]:
# task_1_map ={
#     'NAG' : 0,
#     'CAG' : 1,
#     'OAG' : 2
# }

# cag_data = train[train['Sub-task A'] == 'CAG']
# oag_data = train[train['Sub-task A'] == 'OAG']

gen_data = train[train['Sub-task B'] == 'GEN']

In [36]:
def appendAugData(aug_pipe, df, stopwords, target = None):
  global train
  if target is None:
    raise Exception('Missing Required Argument. target is needed to be a string, found None')

  aug_data = []
  
  def stringApp(string, target, stopwords):
    clean_string = stringCleanerMasker(string, num_mask = 1, stop_words=stopwords, mask_delim = '[MASK]')
    if clean_string == 'INVALID':
      return
    # print("Clean String is : ", clean_string)
    aug_preds = aug_pipe(clean_string)

    for item in aug_preds:
      sentence = item['sequence']
      aug_data.append((sentence, target))

  
  [stringApp(x, y, stopwords) for x, y in zip(df['Text'], df['Sub-task B'])]

  aug_data = pd.DataFrame(aug_data, columns = df.columns)
  train = train.append(aug_data, ignore_index = True)

In [37]:
stop_words = set(nltk.corpus.stopwords.words('english'))

appendAugData(aug_pipe, gen_data, stop_words, target = 'CAG')

# train['Sub-task B'].value_counts()

KeyError: ignored

In [38]:
train.to_csv("TASK_B_train_aug_english.csv", index = False)

In [40]:
train['Sub-task B'].value_counts()

NGEN    3954
GEN     1854
Name: Sub-task B, dtype: int64